In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
import logging
import os
import time

import numpy as np
import matplotlib.pyplot as plt
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim

from lib.dataset_utils import *
from lib.mnist_model import *

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [36]:
exp = 4
model_name = 'mnist_linf_ibp_exp%d.h5' % exp
net = IBPMedium()

In [37]:
# Set all random seeds
seed = 2019
np.random.seed(seed)
torch.manual_seed(seed)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [38]:
# Set up model directory
save_dir = os.path.join(os.getcwd(), 'saved_models/')
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)

net = net.to(device)
# if device == 'cuda':
#     net = torch.nn.DataParallel(net)
#     cudnn.benchmark = True
net.load_state_dict(torch.load(model_path))
# net = net.module
# net = net.basic_net
net.eval()

IBPMedium(
  (conv1): IBPConv(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu1): IBPReLU(inplace=True)
  (conv2): IBPConv(32, 32, kernel_size=(4, 4), stride=(2, 2))
  (relu2): IBPReLU(inplace=True)
  (conv3): IBPConv(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (relu3): IBPReLU(inplace=True)
  (conv4): IBPConv(64, 64, kernel_size=(4, 4), stride=(2, 2))
  (relu4): IBPReLU(inplace=True)
  (fc1): IBPLinear()
  (relu5): IBPReLU(inplace=True)
  (fc2): IBPLinear()
  (relu6): IBPReLU(inplace=True)
  (fc3): IBPLinear()
)

In [7]:
(x_train, y_train), (x_valid, y_valid), (x_test, y_test) = load_mnist_all(
    '/data', val_size=0.1, seed=seed)

In [8]:
def get_input_bound(x, eps):
    x_ub = torch.clamp(x + eps, 0, 1)
    x_lb = torch.clamp(x - eps, 0, 1)
    return x, x_ub, x_lb

In [39]:
eps = 0.1
x_tuple = get_input_bound(x_test.to(device), eps)

with torch.no_grad():
    out = net.get_bound(x_tuple, y_test.to(device))

> /home/research/network-bound/lib/mnist_model.py(231)get_bound()
-> return diff


(Pdb)  c


In [40]:
(out < 0).sum()

tensor(562, device='cuda:0')

In [11]:
eps = 0.1
x_tuple = get_input_bound(x_test.to(device), eps)

with torch.no_grad():
    out = net(x_tuple)

In [12]:
z, z_ub, z_lb = out

In [13]:
eye = torch.eye(z.size(1), device=z.device)
mask = eye[y_test]
z_hat = (1 - mask) * z_ub + mask * z_lb

In [29]:
(z_hat.argmax(1).cpu() != y_test).sum()

tensor(688)

In [35]:
diff = (((z_hat * mask).sum(1) - (z_hat.max(1)[0])) - out).cpu().numpy()
diff[diff != 0]

array([-3.31401825e-04, -4.50730324e-04, -2.44617462e-04, -3.67164612e-04,
       -4.05550003e-04, -2.49385834e-04, -3.98635864e-04, -2.15904236e-01,
       -1.99556351e-04, -2.38895416e-04, -2.43186951e-04, -1.09672546e-03,
       -2.14338303e-04, -1.95741653e-04, -6.15715981e-05, -8.26120377e-05,
       -3.66687775e-04, -1.19018555e-03, -1.22725964e-04, -1.21682882e-04,
       -7.01189041e-04, -1.36137009e-04, -4.24861908e-04, -3.49998474e-04,
       -2.83956528e-04, -6.39796257e-04, -1.67369843e-04, -3.24618816e-03,
       -1.48832798e-04, -4.00543213e-04, -1.53064728e-04, -1.60217285e-04,
       -7.03811646e-04, -1.16825104e-04, -1.97410583e-04, -2.01225281e-04,
       -2.27928162e-04, -2.03251839e-04, -1.57356262e-04, -1.91450119e-04,
       -2.11477280e-04, -2.12028503e-01, -2.46703625e-04, -6.06298447e-04,
       -2.48193741e-04, -3.13520432e-04, -7.71284103e-05, -3.13282013e-04,
       -8.22544098e-04, -4.99725342e-04, -3.47137451e-04, -1.51157379e-04,
       -4.29391861e-04, -

In [24]:
((z_hat * mask).sum(1) - (z_hat.max(1)[0]))[20]

tensor(-1.3773, device='cuda:0')

In [25]:
out[20]

tensor(-1.3768, device='cuda:0')

In [30]:
0.7225 - 2.8149 

-2.0924